# Industrialized Time Series Forecasting: Bidirectional LSTM with Attention

**Author:** Olivier Robert-Duboille
**Date:** 2024-05-22
**Version:** 2.0 (Industrialized)

## 1. Abstract
This notebook presents a production-grade workflow for time series forecasting. Moving beyond simple ARIMA or vanilla LSTM implementations, we construct a robust Deep Learning pipeline featuring:
- **Bidirectional LSTMs** to capture temporal dependencies in both forward and backward directions.
- **Custom Attention Mechanism** to weigh the importance of different time steps in the input sequence.
- **Walk-Forward Validation (TimeSeriesSplit)** for realistic performance estimation.
- **Hyperparameter Tuning** integration (stubbed for KerasTuner).
- **Residual Analysis** to verify statistical assumptions.

## 2. Methodology: The LSTM & Attention Mechanism

### 2.1 Long Short-Term Memory (LSTM)
The LSTM unit mitigates the vanishing gradient problem in RNNs using a gating mechanism. For a given time step $t$, the hidden state $h_t$ and cell state $c_t$ are updated as follows:

$$
\begin{aligned}
f_t &= \sigma(W_f \cdot [h_{t-1}, x_t] + b_f) \quad &(\text{Forget Gate}) \\
i_t &= \sigma(W_i \cdot [h_{t-1}, x_t] + b_i) \quad &(\text{Input Gate}) \\
\tilde{c}_t &= \tanh(W_c \cdot [h_{t-1}, x_t] + b_c) \quad &(\text{Candidate Cell}) \\
c_t &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \quad &(\text{Cell State Update}) \\
o_t &= \sigma(W_o \cdot [h_{t-1}, x_t] + b_o) \quad &(\text{Output Gate}) \\
h_t &= o_t \odot \tanh(c_t) \quad &(\text{Hidden State})
\end{aligned}
$$

### 2.2 Attention Mechanism
To allow the model to focus on specific past time steps that are most relevant for the prediction, we apply an attention mechanism over the LSTM outputs $H = [h_1, h_2, ..., h_T]$.

$$
\begin{aligned}
e_t &= \tanh(W_a h_t + b_a) \\
\alpha_t &= \frac{\exp(e_t)}{\sum_{k=1}^{T} \exp(e_k)} \\
c &= \sum_{t=1}^{T} \alpha_t h_t
\end{aligned}
$$

Where $c$ is the context vector passed to the final dense layers.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Input, Layer, Activation, Dot, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (14, 7)

print(f"TensorFlow Version: {tf.__version__}")

## 3. Data Generation: Simulating Complex Realities
We generate a synthetic time series that mimics complex financial or sensor data. It includes:
1.  **Non-linear Trend**: A logistic growth trend.
2.  **Multi-Seasonality**: Weekly and Monthly cycles.
3.  **Heteroscedastic Noise**: Noise variance that changes over time.
4.  **Structural Breaks**: Sudden shifts in the baseline.

In [ ]:
def generate_complex_series(n_steps=2000):
    t = np.arange(n_steps)
    
    # 1. Non-linear Trend (Logistic-like)
    trend = 100 / (1 + np.exp(-(t - n_steps/2) / 400))
    
    # 2. Multi-Seasonality
    seasonality_weekly = 10 * np.sin(2 * np.pi * t / 7)
    seasonality_monthly = 20 * np.sin(2 * np.pi * t / 30)
    
    # 3. Heteroscedastic Noise (Variance increases with time)
    noise_level = 2 + (t / n_steps) * 5
    noise = np.random.normal(0, 1, n_steps) * noise_level
    
    # 4. Structural Break (Shift at t=1500)
    break_point = 1500
    structural_break = np.zeros(n_steps)
    structural_break[break_point:] = 30
    
    series = trend + seasonality_weekly + seasonality_monthly + noise + structural_break
    
    date_range = pd.date_range(start='2018-01-01', periods=n_steps, freq='D')
    return pd.DataFrame({'value': series}, index=date_range)

df = generate_complex_series()

plt.figure(figsize=(16, 6))
plt.plot(df.index, df['value'], label='Complex Series', linewidth=1)
plt.title('Synthetic Data with Trend, Seasonality, Heteroscedasticity, and Breaks', fontsize=16)
plt.legend()
plt.show()

## 4. Feature Engineering & Preprocessing
Raw time series are rarely sufficient. We engineer features to help the LSTM converge faster:
-   **Lag Features**: Values at $t-7, t-30$ (capturing seasonality).
-   **Rolling Statistics**: Rolling mean and std dev to capture local trends/volatility.
-   **Robust Scaling**: Using median and IQR to be robust against outliers (noise).

In [ ]:
def engineer_features(df):
    data = df.copy()
    
    # Lag features
    data['lag_7'] = data['value'].shift(7)
    data['lag_30'] = data['value'].shift(30)
    
    # Rolling statistics
    data['rolling_mean_7'] = data['value'].rolling(window=7).mean()
    data['rolling_std_7'] = data['value'].rolling(window=7).std()
    
    data.dropna(inplace=True)
    return data

df_features = engineer_features(df)
print(f"Data shape after feature engineering: {df_features.shape}")

# Preprocessing
scaler = RobustScaler()
scaled_data = scaler.fit_transform(df_features)

# Sequence Generation
def create_sequences(data, seq_length, forecast_horizon=1):
    X, y = [], []
    for i in range(len(data) - seq_length - forecast_horizon + 1):
        X.append(data[i:(i + seq_length)])
        # Target: Predicting the 'value' column (index 0) 'forecast_horizon' steps ahead
        y.append(data[i + seq_length + forecast_horizon - 1, 0]) 
    return np.array(X), np.array(y)

SEQ_LENGTH = 60
X, y = create_sequences(scaled_data, SEQ_LENGTH)

print(f"Input Shape: {X.shape} (Samples, Timesteps, Features)")
print(f"Target Shape: {y.shape}")

## 5. Model Architecture: BiLSTM + Attention
We implement a custom Attention layer inheriting from `tf.keras.layers.Layer`.

In [ ]:
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', 
                                 shape=(input_shape[-1], 1),
                                 initializer='normal')
        self.b = self.add_weight(name='attention_bias', 
                                 shape=(input_shape[1], 1),
                                 initializer='zeros')
        super(Attention, self).build(input_shape)

    def call(self, x):
        # x shape: (batch_size, time_steps, features)
        # e = tanh(dot(x, W) + b)
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        
        # alpha = softmax(e)
        # Remove dimension of size 1
        e = tf.squeeze(e, axis=-1) 
        alpha = tf.keras.backend.softmax(e)
        
        # context = sum(alpha * x)
        alpha = tf.expand_dims(alpha, axis=-1)
        context = x * alpha
        context = tf.keras.backend.sum(context, axis=1)
        
        return context

def build_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Bidirectional LSTM to capture past and future context within the window
    lstm_out = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    lstm_out = Dropout(0.3)(lstm_out)
    
    # Second LSTM layer
    lstm_out = Bidirectional(LSTM(32, return_sequences=True))(lstm_out)
    lstm_out = Dropout(0.3)(lstm_out)
    
    # Attention Layer
    attention_out = Attention()(lstm_out)
    
    # Dense Layers
    x = Dense(32, activation='relu')(attention_out)
    x = Dropout(0.2)(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

model_example = build_model((SEQ_LENGTH, X.shape[2]))
model_example.summary()

## 6. Walk-Forward Validation (Backtesting)
Standard K-Fold is invalid for time series because it shuffles data. We use `TimeSeriesSplit` to respect temporal order.

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)

fold = 1
rmse_scores = []

for train_index, test_index in tscv.split(X):
    print(f"\n--- Fold {fold} ---")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = build_model((SEQ_LENGTH, X.shape[2]))
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop, reduce_lr],
        verbose=0  # Silent training for cleanliness
    )
    
    # Evaluate
    preds = model.predict(X_test, verbose=0)
    
    # Inverse Transform (trickier with RobustScaler on multiple features)
    # We constructed X with multiple features, but y is just the first column (index 0)
    # To inverse transform, we need a dummy array of the same shape as original input
    
    def inverse_transform_target(scaler, y_scaled, n_features):
        dummy = np.zeros((len(y_scaled), n_features))
        dummy[:, 0] = y_scaled.flatten()
        return scaler.inverse_transform(dummy)[:, 0]

    y_test_inv = inverse_transform_target(scaler, y_test, X.shape[2])
    preds_inv = inverse_transform_target(scaler, preds, X.shape[2])
    
    rmse = np.sqrt(mean_squared_error(y_test_inv, preds_inv))
    print(f"Fold {fold} RMSE: {rmse:.4f}")
    rmse_scores.append(rmse)
    
    # Plot last fold
    if fold == 3:
        plt.figure(figsize=(12, 6))
        plt.plot(y_test_inv, label='Actual', color='black', alpha=0.7)
        plt.plot(preds_inv, label='Predicted', color='teal', alpha=0.7)
        plt.title(f'Forecast vs Actual (Fold {fold})')
        plt.legend()
        plt.show()
        
    fold += 1

print(f"\nAverage RMSE: {np.mean(rmse_scores):.4f}")

## 7. Residual Analysis
A good model should have residuals (errors) that resemble white noise (normally distributed, no autocorrelation).

In [ ]:
residuals = y_test_inv - preds_inv

fig, ax = plt.subplots(1, 2, figsize=(16, 5))

# Distribution
sns.histplot(residuals, kde=True, ax=ax[0], color='salmon')
ax[0].set_title('Residual Distribution')
ax[0].set_xlabel('Error')

# Autocorrelation
pd.plotting.autocorrelation_plot(residuals, ax=ax[1])
ax[1].set_title('Residual Autocorrelation')

plt.show()

print(f"Mean Residual: {np.mean(residuals):.4f}")
print(f"Std Residual: {np.std(residuals):.4f}")